## Buy Now, Pay Later Project
### MAST30034: Applied Data Science 
#### Notebook 1: Preprocessing Data

In [1]:
# create modeling spark session
from pyspark.sql import SparkSession

spark = (
    SparkSession.builder.appName('Project 2')
    .config('spark.sql.repl.eagerEval.enabled', True) 
    .config('spark.sql.parquet.cacheMetadata', 'true')
    .config("spark.sql.session.timeZone", "Etc/UTC")
    .config("spark.executor.memory", "8g")
    .config("spark.driver.memory", "8g")
    .getOrCreate()
)

22/09/27 19:52:12 WARN Utils: Your hostname, DESKTOP-G0KJF2G resolves to a loopback address: 127.0.1.1; using 172.31.175.79 instead (on interface eth0)
22/09/27 19:52:12 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/09/27 19:52:13 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


#### Merchant Dataset:

In [2]:
# read merchants data
merchants = spark.read.parquet('../data/tables/tbl_merchants.parquet')
merchants.limit(5)

name,tags,merchant_abn
Felis Limited,"((furniture, home...",10023283211
Arcu Ac Orci Corp...,"([cable, satellit...",10142254217
Nunc Sed Company,"([jewelry, watch,...",10165489824
Ultricies Digniss...,"([wAtch, clock, a...",10187291046
Enim Condimentum PC,([music shops - m...,10192359162


In [3]:
# In the coloumn "tags", tags, renueve level and "take_rate" are seperated by either "), (" or "], [", replace these by ### for splitting
from pyspark.sql import functions as F
merchants = merchants.withColumn("tags",F.regexp_replace(F.regexp_replace(
    F.col("tags"),"\){1},{1}\s?\(","###"),"\]{1},{1}\s?\[","###"))


In [4]:
# check if correctly replaced
merchants.select("tags").show(5, False)

+---------------------------------------------------------------------------------------------------------------+
|tags                                                                                                           |
+---------------------------------------------------------------------------------------------------------------+
|((furniture, home furnishings and equipment shops, and manufacturers, except appliances###e###take rate: 0.18))|
|([cable, satellite, and otHer pay television and radio services###b###take rate: 4.22])                        |
|([jewelry, watch, clock, and silverware shops###b###take rate: 4.40])                                          |
|([wAtch, clock, and jewelry repair shops###b###take rate: 3.29])                                               |
|([music shops - musical instruments, pianos, and sheet music###a###take rate: 6.33])                           |
+---------------------------------------------------------------------------------------

In [4]:
# split tags data to three columns, denoted as business_area, revenue_level and take_rate
split_merchants = merchants.withColumn("business_area", F.split(F.col("tags"), "###").getItem(0))\
                           .withColumn("revenue_level", F.split(F.col("tags"), "###").getItem(1))\
                           .withColumn("take_rate", F.split(F.col("tags"), "###").getItem(2))

In [6]:
split_merchants.limit(5)

name,tags,merchant_abn,business_area,revenue_level,take_rate
Felis Limited,"((furniture, home...",10023283211,"((furniture, home...",e,take rate: 0.18))
Arcu Ac Orci Corp...,"([cable, satellit...",10142254217,"([cable, satellit...",b,take rate: 4.22])
Nunc Sed Company,"([jewelry, watch,...",10165489824,"([jewelry, watch,...",b,take rate: 4.40])
Ultricies Digniss...,"([wAtch, clock, a...",10187291046,"([wAtch, clock, a...",b,take rate: 3.29])
Enim Condimentum PC,([music shops - m...,10192359162,([music shops - m...,a,take rate: 6.33])


In [5]:
# remove remaining ()[] punctuation in column business_area and take_rate

# obtain take rate in numeric form from take_rate column, 
# in the form of take rate: d.dd, therefore item 1 is the numeric value

# convert string in business_area to lower form

# convert string in revenue_level to lower form, just in case if there's a typo

curated_merchant = split_merchants\
            .withColumn("business_area", F.regexp_replace("business_area", "[\[\]\(\)]", ""))\
            .withColumn("take_rate", F.regexp_replace("take_rate", "[\[\]\(\)]", ""))\
            .withColumn("take_rate", F.split(F.col("take_rate"), ":").getItem(1))\
            .withColumn("business_area", F.lower(F.col('business_area')))\
            .withColumn("revenue_level", F.lower(F.col('revenue_level')))


final_merchant = curated_merchant.drop('tags')

In [8]:
final_merchant.limit(5)

name,merchant_abn,business_area,revenue_level,take_rate
Felis Limited,10023283211,"furniture, home f...",e,0.18
Arcu Ac Orci Corp...,10142254217,"cable, satellite,...",b,4.22
Nunc Sed Company,10165489824,"jewelry, watch, c...",b,4.40
Ultricies Digniss...,10187291046,"watch, clock, and...",b,3.29
Enim Condimentum PC,10192359162,music shops - mus...,a,6.33


In [9]:
# check if revenue_level match levels (5 types) and count for the number of merchants in each level
final_merchant.groupby('revenue_level').count()


revenue_level,count
e,53
d,98
c,922
b,1351
a,1602


In [10]:
# check for different business area count 
final_merchant.groupby('business_area').count().sort("business_area")


business_area,count
antique shops - ...,3
antique shops - s...,2
antique shops - s...,124
art dealers and ...,1
art dealers and g...,111
artist supply an...,3
artist supply and...,2
artist supply and...,1
artist supply and...,187
bicycle shops - ...,1


Noticed that there are several entries which have the same beginning of business area, to check if they are indeed the same, we convert the "business_area" as list and print them as follows

In [6]:
final_merchant.groupby('business_area').count().sort("business_area").select("business_area").collect()

[Row(business_area='antique shops -  sales, repairs, and restoration services'),
 Row(business_area='antique shops - sales,  repairs, and restoration services'),
 Row(business_area='antique shops - sales, repairs, and restoration services'),
 Row(business_area='art dealers and  galleries'),
 Row(business_area='art dealers and galleries'),
 Row(business_area='artist supply  and craft shops'),
 Row(business_area='artist supply and  craft shops'),
 Row(business_area='artist supply and craft  shops'),
 Row(business_area='artist supply and craft shops'),
 Row(business_area='bicycle  shops - sales and service'),
 Row(business_area='bicycle shops  - sales and service'),
 Row(business_area='bicycle shops -  sales and service'),
 Row(business_area='bicycle shops - sales and  service'),
 Row(business_area='bicycle shops - sales and service'),
 Row(business_area='books,  periodicals, and newspapers'),
 Row(business_area='books, periodicals,  and newspapers'),
 Row(business_area='books, periodical

In [7]:
# The concepts are indeed the same but with some extra spaces. so remove extra spaces in "business_area"
final_merchant = final_merchant.withColumn("business_area", F.regexp_replace("business_area", "\s+", " "))

In [13]:
# check for different business area count again
final_merchant.groupby('business_area').count().sort("business_area")

business_area,count
antique shops - s...,129
art dealers and g...,112
artist supply and...,193
bicycle shops - s...,170
"books, periodical...",164
"cable, satellite,...",175
computer programm...,191
"computers, comput...",181
digital goods: bo...,195
"equipment, tool, ...",134


In [8]:
# collect all merchant business area
final_merchant.groupby('business_area').count().select("business_area").collect()

[Row(business_area='opticians, optical goods, and eyeglasses'),
 Row(business_area='watch, clock, and jewelry repair shops'),
 Row(business_area='computer programming , data processing, and integrated systems design services'),
 Row(business_area='digital goods: books, movies, music'),
 Row(business_area='books, periodicals, and newspapers'),
 Row(business_area='florists supplies, nursery stock, and flowers'),
 Row(business_area='art dealers and galleries'),
 Row(business_area='antique shops - sales, repairs, and restoration services'),
 Row(business_area='gift, card, novelty, and souvenir shops'),
 Row(business_area='equipment, tool, furniture, and appliance rent al and leasing'),
 Row(business_area='cable, satellite, and other pay television and radio services'),
 Row(business_area='tent and awning shops'),
 Row(business_area='artist supply and craft shops'),
 Row(business_area='stationery, office supplies and printing and writing paper'),
 Row(business_area='furniture, home furnishi

#### Consumer Dataset:

In [9]:
consumer = spark.read.parquet('../data/tables/consumer_user_details.parquet')
consumer.limit(10)

user_id,consumer_id
1,1195503
2,179208
3,1194530
4,154128
5,712975
6,407340
7,511685
8,448088
9,650435
10,1058499


In [16]:
# count the number of consumer
consumer.count()

499999

In [10]:
consumer_detail = spark.read.option('sep', "|").csv('../data/tables/tbl_consumer.csv', 
                                                    header = True)
consumer_detail.limit(10)

name,address,state,postcode,gender,consumer_id
Yolanda Williams,413 Haney Gardens...,WA,6935,Female,1195503
Mary Smith,3764 Amber Oval,NSW,2782,Female,179208
Jill Jones MD,40693 Henry Greens,NT,862,Female,1194530
Lindsay Jimenez,00653 Davenport C...,NSW,2780,Female,154128
Rebecca Blanchard,9271 Michael Mano...,WA,6355,Female,712975
Karen Chapman,2706 Stewart Oval...,NSW,2033,Female,407340
Andrea Jones,122 Brandon Cliff,QLD,4606,Female,511685
Stephen Williams,6804 Wright Crest...,WA,6056,Male,448088
Stephanie Reyes,5813 Denise Land ...,NSW,2482,Female,650435
Jillian Gonzales,461 Ryan Common S...,VIC,3220,Female,1058499


In [11]:
# add the consumer details onto consumer dataframe
consumer = consumer.join(consumer_detail,on="consumer_id")

In [19]:
consumer.show(10)

+-----------+-------+-----------------+--------------------+-----+--------+------+
|consumer_id|user_id|             name|             address|state|postcode|gender|
+-----------+-------+-----------------+--------------------+-----+--------+------+
|    1195503|      1| Yolanda Williams|413 Haney Gardens...|   WA|    6935|Female|
|     179208|      2|       Mary Smith|     3764 Amber Oval|  NSW|    2782|Female|
|    1194530|      3|    Jill Jones MD|  40693 Henry Greens|   NT|     862|Female|
|     154128|      4|  Lindsay Jimenez|00653 Davenport C...|  NSW|    2780|Female|
|     712975|      5|Rebecca Blanchard|9271 Michael Mano...|   WA|    6355|Female|
|     407340|      6|    Karen Chapman|2706 Stewart Oval...|  NSW|    2033|Female|
|     511685|      7|     Andrea Jones|   122 Brandon Cliff|  QLD|    4606|Female|
|     448088|      8| Stephen Williams|6804 Wright Crest...|   WA|    6056|  Male|
|     650435|      9|  Stephanie Reyes|5813 Denise Land ...|  NSW|    2482|Female|
|   

#### Transaction dataset:

In [12]:
# read transaction data
transaction_210228 = spark.read.parquet("../data/tables/transactions_20210228_20210827_snapshot")
transaction_210828 = spark.read.parquet("../data/tables/transactions_20210828_20220227_snapshot")
transaction_220228 = spark.read.parquet("../data/tables/transactions_20220228_20220828_snapshot")
transaction = transaction_210228.union(transaction_210828)
transaction = transaction.union(transaction_220228)

In [21]:
# count total number of transactions
transaction.count()

14195505

In [22]:
transaction_220228.limit(10)

user_id,merchant_abn,dollar_value,order_id,order_datetime
11139,96152467973,16.213590228273233,785b0080-9e4b-471...,2022-08-20
1,98973094975,86.97955945703498,2560f7b0-ee5d-4b3...,2022-08-20
11139,56762458844,31.513502323509194,0311717b-8b5b-410...,2022-08-20
1,89502033586,124.18468694868491,f8891626-f098-45b...,2022-08-20
11139,96161808980,61.620445567668966,d90a421f-f1da-4bf...,2022-08-20
2,72472909171,32.26524985312485,523e0403-b677-450...,2022-08-20
11139,91923722701,11.331586767322225,f45a842b-0366-41d...,2022-08-20
3,46380096952,119.80011239189334,58d0f423-037c-43f...,2022-08-20
11140,79283124876,198.13027742225435,60b12d41-41d6-4c1...,2022-08-20
4,67202032418,206.20865323560022,64a05a23-a078-481...,2022-08-20


In [13]:
# check if there are any null values in the dataframe
transaction.filter(F.col("order_datetime").isNull())

user_id,merchant_abn,dollar_value,order_id,order_datetime


In [14]:
# merge all three dataset

consumer_transaction = consumer.join(transaction,['user_id'],how='inner')
full_transaction_dataset = consumer_transaction.join(final_merchant,['merchant_abn'],how='left')

In [25]:
full_transaction_dataset.count()

14195505

In [15]:
# count the total transactions for each merchant
merchant_count = transaction.groupBy(F.col('merchant_abn')).count()

In [16]:
merchant_count.orderBy(F.col('count').asc())

merchant_abn,count
62688594508,1
10404542215,1
86868464441,1
39150153670,1
80664157260,1
37145789569,1
93267734067,1
47047735645,1
99989036621,1
67264251405,1


In [28]:
# check with mechant 62688594508 as it has only 1 transaction record
final_merchant[final_merchant.merchant_abn=='62688594508'].collect()

[Row(name='Lobortis Nisi Associates', merchant_abn=62688594508, business_area='jewelry, watch, clock, and silverware shops', revenue_level='d', take_rate=' 0.88')]

In [17]:
transaction[transaction.merchant_abn=='62688594508'].collect()

[Row(user_id=17098, merchant_abn=62688594508, dollar_value=21112.567917387372, order_id='82f1625f-00f4-4516-b7cc-155badc6431f', order_datetime=datetime.date(2021, 3, 20))]

#### Census:

In [54]:
# read census dataset
census = spark.read.csv('../data/tables/censusData/2021 Census GCP Postal Areas for AUS/2021Census_G02_AUST_POA.csv', header = True)
census.limit(5)

POA_CODE_2021,Median_age_persons,Median_mortgage_repay_monthly,Median_tot_prsnl_inc_weekly,Median_rent_weekly,Median_tot_fam_inc_weekly,Average_num_psns_per_bedroom,Median_tot_hhd_inc_weekly,Average_household_size
POA2000,32,2800,941,625,2367,1.3,2225,2.1
POA2007,30,2500,772,500,2197,1.2,1805,2.1
POA2008,28,2600,860,525,2453,1.2,1746,1.9
POA2009,37,2800,1297,580,3035,1.1,2422,2.1
POA2010,36,2900,1479,550,3709,1.1,2297,1.7


In [60]:
census_state = spark.read.csv('../data/tables/censusData/2021 Census GCP States and Territories for AUS/2021Census_G02_AUST_STE.csv', header = True)
census_state.limit(10)

STE_CODE_2021,Median_age_persons,Median_mortgage_repay_monthly,Median_tot_prsnl_inc_weekly,Median_rent_weekly,Median_tot_fam_inc_weekly,Average_num_psns_per_bedroom,Median_tot_hhd_inc_weekly,Average_household_size
1,39,2167,813,420,2185,0.9,1829,2.6
2,38,1859,803,370,2136,0.8,1759,2.5
3,38,1733,787,365,2024,0.8,1675,2.5
4,41,1500,734,300,1889,0.8,1455,2.4
5,38,1842,848,340,2214,0.8,1815,2.5
6,42,1313,701,290,1720,0.8,1358,2.4
7,33,2000,936,325,2213,1,2061,2.8
8,35,2080,1203,450,2872,0.8,2373,2.5
9,43,1300,829,200,1890,0.9,1497,2.4


In [19]:
census = census.withColumn("postcode", F.regexp_replace("POA_CODE_2021", "POA", ""))
census.limit(5)

POA_CODE_2021,Median_age_persons,Median_mortgage_repay_monthly,Median_tot_prsnl_inc_weekly,Median_rent_weekly,Median_tot_fam_inc_weekly,Average_num_psns_per_bedroom,Median_tot_hhd_inc_weekly,Average_household_size,postcode
POA2000,32,2800,941,625,2367,1.3,2225,2.1,2000
POA2007,30,2500,772,500,2197,1.2,1805,2.1,2007
POA2008,28,2600,860,525,2453,1.2,1746,1.9,2008
POA2009,37,2800,1297,580,3035,1.1,2422,2.1,2009
POA2010,36,2900,1479,550,3709,1.1,2297,1.7,2010


#### Fraud probability dataset:

Probability of fraud transactions are provided for both merchant and customers with given merchant abn and consumer user id, with specific date of transaction. 

In [20]:
merchant_fraud = spark.read.csv('../data/tables/merchant_fraud_probability.csv', 
                                                    header = True)

# rename fraud_probability so it's distinct from consumer's
merchant_fraud = merchant_fraud.withColumnRenamed("fraud_probability","fraud_prob_merch")
merchant_fraud.limit(5)

merchant_abn,order_datetime,fraud_prob_merch
19492220327,2021-11-28,44.403658647495355
31334588839,2021-10-02,42.75530083865367
19492220327,2021-12-22,38.867790051131095
82999039227,2021-12-19,94.1347004808891
90918180829,2021-09-02,43.32551731714902


In [21]:
consumer_fraud = spark.read.csv('../data/tables/consumer_fraud_probability.csv', 
                                                    header = True)

consumer_fraud = consumer_fraud.withColumnRenamed("fraud_probability","fraud_prob_cons")
consumer_fraud.limit(5)

user_id,order_datetime,fraud_prob_cons
6228,2021-12-19,97.6298077657765
21419,2021-12-10,99.24738020302328
5606,2021-10-17,84.05825045251777
3101,2021-04-17,91.42192091901347
22239,2021-10-19,94.70342477508035


In [22]:
consumer_fraud.orderBy(F.col('fraud_prob_cons').desc())

user_id,order_datetime,fraud_prob_cons
21419,2021-12-10,99.24738020302328
21419,2021-12-10,99.24738020302328
6228,2021-12-19,97.6298077657765
6228,2021-12-19,97.6298077657765
22239,2021-10-19,94.70342477508035
22239,2021-10-19,94.70342477508035
8347,2021-10-06,92.99138306039121
8347,2021-10-06,92.99138306039121
17900,2022-02-25,92.73262811161372
17900,2022-02-25,92.73262811161372


We can see that there are some duplicate rows in fraud dataframe, remove duplicated rows

In [23]:
consumer_fraud = consumer_fraud.distinct()

### Merge all dataframe

In [24]:
transaction.groupby('merchant_abn').count().count()

4422

In [37]:
final_merchant.count()

4026

It is found that there are transactions in the dataset that is not done in the merchants in provided merchant dataset, we need to merge transactions with merchant to remove merchants that are not included

In [25]:
trans_merch = transaction.join(final_merchant, on = 'merchant_abn')

In [39]:
trans_merch.count()

13614675

In [40]:
# check if number of disinct merchants match merchant data
trans_merch.groupby('merchant_abn').count().count()

4026

Merchants that are not included are successfully removed from transaction dataset. 

Then merge the dataset with consumer data. We don't need consumer's name and address as they the address is fake and name does not provide general information.  

In [26]:
consumer = consumer.drop("name","address")

In [27]:
trans_merch_cons = trans_merch.join(consumer, on = 'user_id')

In [43]:
trans_merch_cons.limit(5)

user_id,merchant_abn,dollar_value,order_id,order_datetime,name,business_area,revenue_level,take_rate,consumer_id,state,postcode,gender
19,94493496784,121.62828258168003,ff949237-bd02-40b...,2021-08-21,Dictum Phasellus ...,"gift, card, novel...",a,5.65,1226530,TAS,7276,Female
19,79479365013,398.60394093957234,d38ba70d-0b27-48e...,2021-08-19,Facilisis Non Inc.,florists supplies...,a,6.67,1226530,TAS,7276,Female
19,63105795563,423.2372162036774,a4d755ef-5368-40c...,2021-08-19,Aliquet Consulting,artist supply and...,a,5.94,1226530,TAS,7276,Female
19,57900494384,13.411325379072077,52c06ef4-29df-4b7...,2021-08-22,Porttitor Tellus ...,tent and awning s...,a,6.39,1226530,TAS,7276,Female
19,80682333501,1.8745367326734035,af78a3a5-cf65-4f3...,2021-08-22,Orci Corp.,florists supplies...,b,4.88,1226530,TAS,7276,Female


In [44]:
trans_merch_cons.count()

13614675

Now join transaction dataframe with fraud probability data

In [28]:
# join postcode with the full dataset
data_with_fraud = trans_merch_cons.join(consumer_fraud, on = ["user_id","order_datetime"], how = "left")

In [29]:
data_with_fraud.limit(20)

user_id,order_datetime,merchant_abn,dollar_value,order_id,name,business_area,revenue_level,take_rate,consumer_id,state,postcode,gender,fraud_prob_cons
19,2021-08-21,94493496784,121.62828258168003,ff949237-bd02-40b...,Dictum Phasellus ...,"gift, card, novel...",a,5.65,1226530,TAS,7276,Female,null
19,2021-08-19,79479365013,398.60394093957234,d38ba70d-0b27-48e...,Facilisis Non Inc.,florists supplies...,a,6.67,1226530,TAS,7276,Female,null
19,2021-08-19,63105795563,423.2372162036774,a4d755ef-5368-40c...,Aliquet Consulting,artist supply and...,a,5.94,1226530,TAS,7276,Female,null
19,2021-08-22,57900494384,13.411325379072077,52c06ef4-29df-4b7...,Porttitor Tellus ...,tent and awning s...,a,6.39,1226530,TAS,7276,Female,null
19,2021-08-22,80682333501,1.8745367326734035,af78a3a5-cf65-4f3...,Orci Corp.,florists supplies...,b,4.88,1226530,TAS,7276,Female,null
19,2021-08-14,46622394552,918.8573491448071,66f7946b-ab33-476...,Laoreet Libero Et...,"stationery, offic...",a,6.76,1226530,TAS,7276,Female,null
19,2021-07-15,86578477987,21.251020797421425,2b074c63-33f9-444...,Leo In Consulting,"watch, clock, and...",a,6.43,1226530,TAS,7276,Female,null
19,2021-08-16,29625244690,234.9066053905527,26d28826-acab-47e...,Id Enim Inc.,"books, periodical...",a,6.01,1226530,TAS,7276,Female,null
19,2021-07-23,32361057556,83.96113155010558,2b0769e4-af43-4ae...,Orci In Consequat...,"gift, card, novel...",a,6.61,1226530,TAS,7276,Female,null
19,2021-05-21,61997656782,315.7344805490105,8a42dafb-f515-4e5...,Mauris Erat Ltd,motor vehicle sup...,b,4.40,1226530,TAS,7276,Female,null


This will result in null values for fraud_prob_cons if consumer ID and datatime are not in fraud probability dataframe, fill the null values by 0. Spark dataframe cannot fill null values in a column with float variables, hence we need to convert float to integer for spark to fill in null values

In [30]:
full_dataset_filna = data_with_fraud.withColumn("fraud_prob_cons", F.col("fraud_prob_cons").cast("int")).fillna(0)


In [48]:
full_dataset_filna.limit(5)

user_id,order_datetime,merchant_abn,dollar_value,order_id,name,business_area,revenue_level,take_rate,consumer_id,state,postcode,gender,fraud_prob_cons
19,2021-08-21,94493496784,121.62828258168003,ff949237-bd02-40b...,Dictum Phasellus ...,"gift, card, novel...",a,5.65,1226530,TAS,7276,Female,0
19,2021-08-19,79479365013,398.60394093957234,d38ba70d-0b27-48e...,Facilisis Non Inc.,florists supplies...,a,6.67,1226530,TAS,7276,Female,0
19,2021-08-19,63105795563,423.2372162036774,a4d755ef-5368-40c...,Aliquet Consulting,artist supply and...,a,5.94,1226530,TAS,7276,Female,0
19,2021-08-22,57900494384,13.411325379072077,52c06ef4-29df-4b7...,Porttitor Tellus ...,tent and awning s...,a,6.39,1226530,TAS,7276,Female,0
19,2021-08-22,80682333501,1.8745367326734035,af78a3a5-cf65-4f3...,Orci Corp.,florists supplies...,b,4.88,1226530,TAS,7276,Female,0


In [31]:
# select transactions with fraud probability of less than 50 to be valid transactions
data_remove_fraud = full_dataset_filna.filter(full_dataset_filna.fraud_prob_cons<50)

In [50]:
data_remove_fraud.count() - full_dataset_filna.count()

-1063

In [36]:
# adding lead zeros for post code
from pyspark.sql.functions import lpad
data_remove_fraud = data_remove_fraud.withColumn('postcode',lpad(data_remove_fraud['postcode'],4,'0'))

Removed 1063 fraud transactions with fraud probability over 50%

Then we should merge the dataset with census data

In [37]:
full_dataset = data_remove_fraud.join(census, on = ["postcode"], how = "left")

In [35]:
from pyspark.sql.functions import lpad
full_dataset =  full_dataset.withColumn('postcode',lpad(full_dataset['postcode'],4,'0'))

In [46]:
full_dataset.limit(5)

postcode,user_id,order_datetime,merchant_abn,dollar_value,order_id,name,business_area,revenue_level,take_rate,consumer_id,state,gender,fraud_prob_cons,POA_CODE_2021,Median_age_persons,Median_mortgage_repay_monthly,Median_tot_prsnl_inc_weekly,Median_rent_weekly,Median_tot_fam_inc_weekly,Average_num_psns_per_bedroom,Median_tot_hhd_inc_weekly,Average_household_size
7276,19,2021-08-21,94493496784,121.62828258168003,ff949237-bd02-40b...,Dictum Phasellus ...,"gift, card, novel...",a,5.65,1226530,TAS,Female,0,POA7276,46,1221,689,235,1738,0.8,1380,2.3
7276,19,2021-08-19,79479365013,398.60394093957234,d38ba70d-0b27-48e...,Facilisis Non Inc.,florists supplies...,a,6.67,1226530,TAS,Female,0,POA7276,46,1221,689,235,1738,0.8,1380,2.3
7276,19,2021-08-19,63105795563,423.2372162036774,a4d755ef-5368-40c...,Aliquet Consulting,artist supply and...,a,5.94,1226530,TAS,Female,0,POA7276,46,1221,689,235,1738,0.8,1380,2.3
7276,19,2021-08-22,57900494384,13.411325379072077,52c06ef4-29df-4b7...,Porttitor Tellus ...,tent and awning s...,a,6.39,1226530,TAS,Female,0,POA7276,46,1221,689,235,1738,0.8,1380,2.3
7276,19,2021-08-22,80682333501,1.8745367326734035,af78a3a5-cf65-4f3...,Orci Corp.,florists supplies...,b,4.88,1226530,TAS,Female,0,POA7276,46,1221,689,235,1738,0.8,1380,2.3


In [56]:
census.count()

2643

In [73]:
full_dataset = full_dataset.drop("POA_CODE_2021")

In [80]:
dataset_with_null = full_dataset.filter(F.col('Median_tot_prsnl_inc_weekly').isNull()).groupby("postcode").count()

In [69]:
full_dataset = full_dataset.filter(F.col('Median_tot_prsnl_inc_weekly').isNotNull())

In [81]:
dataset_with_null = dataset_with_null.withColumn(
    'STE_CODE_2021',
    F.when(((F.col("postcode") >= 1000) & (F.col("postcode") <= 2599 )) | ((F.col("postcode") >= 2619) & (F.col("postcode") <= 2898 )) | ((F.col("postcode") >= 2921) & (F.col("postcode") <= 2999 )), 1)\

    .when(((F.col("postcode") >= '0200') & (F.col("postcode") <= '0299' )) | ((F.col("postcode") >= 2600) & (F.col("postcode") <= 2618 )) | ((F.col("postcode") >= 2900) & (F.col("postcode") <= 2920 )), 8)\

    .when(((F.col("postcode") >= 3000) & (F.col("postcode") <= 3999 )) | ((F.col("postcode") >= 8000) & (F.col("postcode") <= 8999 )),2)\

    .when(((F.col("postcode") >= 4000) & (F.col("postcode") <= 4999 )) | ((F.col("postcode") >= 9000) & (F.col("postcode") <= 9999 )),3)\

    .when(((F.col("postcode") >= 5000) & (F.col("postcode") <= 5999  )),4)\

    .when(((F.col("postcode") >= 6000) & (F.col("postcode") <= 6797 )) | ((F.col("postcode") >= 6800) & (F.col("postcode") <= 6999 )),5)\

    .when(((F.col("postcode") >= 7000) & (F.col("postcode") <= 8000 )), 6)\

    .otherwise(7)
)

In [82]:
dataset_with_null = dataset_with_null.join(census_state, ['STE_CODE_2021']).drop("STE_CODE_2021",'count')

In [83]:
dataset_with_null = data_remove_fraud.join(dataset_with_null, on = ["postcode"], how = "right")

In [79]:
dataset_with_null.limit(1)

postcode,user_id,order_datetime,merchant_abn,dollar_value,order_id,name,business_area,revenue_level,take_rate,consumer_id,state,gender,fraud_prob_cons,count,Median_age_persons,Median_mortgage_repay_monthly,Median_tot_prsnl_inc_weekly,Median_rent_weekly,Median_tot_fam_inc_weekly,Average_num_psns_per_bedroom,Median_tot_hhd_inc_weekly,Average_household_size
0862,3,2021-08-20,60956456424,136.6785200286976,0f09c5a5-784e-447...,Ultricies Digniss...,"gift, card, novel...",b,4.69,1194530,NT,Female,0,null,null,null,null,null,null,null,null,null


In [84]:
dataset_with_null.dtypes

[('postcode', 'string'),
 ('user_id', 'bigint'),
 ('order_datetime', 'date'),
 ('merchant_abn', 'bigint'),
 ('dollar_value', 'double'),
 ('order_id', 'string'),
 ('name', 'string'),
 ('business_area', 'string'),
 ('revenue_level', 'string'),
 ('take_rate', 'string'),
 ('consumer_id', 'bigint'),
 ('state', 'string'),
 ('gender', 'string'),
 ('fraud_prob_cons', 'int'),
 ('Median_age_persons', 'string'),
 ('Median_mortgage_repay_monthly', 'string'),
 ('Median_tot_prsnl_inc_weekly', 'string'),
 ('Median_rent_weekly', 'string'),
 ('Median_tot_fam_inc_weekly', 'string'),
 ('Average_num_psns_per_bedroom', 'string'),
 ('Median_tot_hhd_inc_weekly', 'string'),
 ('Average_household_size', 'string')]

In [86]:
full_dataset = full_dataset.union(dataset_with_null)

In [87]:
full_dataset.filter(F.col('Median_tot_prsnl_inc_weekly').isNull()).groupby("postcode").count().count()

0

In [60]:
full_dataset = full_dataset.drop("POA_CODE_2021", "order_id")

In [63]:
full_dataset.count()

13613612

In [61]:
full_dataset.limit(5)

postcode,user_id,order_datetime,merchant_abn,dollar_value,name,business_area,revenue_level,take_rate,consumer_id,state,gender,fraud_prob_cons,Median_age_persons,Median_mortgage_repay_monthly,Median_tot_prsnl_inc_weekly,Median_rent_weekly,Median_tot_fam_inc_weekly,Average_num_psns_per_bedroom,Median_tot_hhd_inc_weekly,Average_household_size
4606,7,2021-08-20,33064796871,373.0873675184212,Curabitur Massa C...,computer programm...,b,3.75,511685,QLD,Female,0,52,1073,536,220,1343,0.7,1003,2.2
4606,7,2021-08-20,68435002949,232.5364986739752,Aliquam Eu Inc.,artist supply and...,a,6.65,511685,QLD,Female,0,52,1073,536,220,1343,0.7,1003,2.2
4606,7,2021-08-20,41944909975,30.910755230234322,Et Nunc Consulting,"books, periodical...",e,0.16,511685,QLD,Female,0,52,1073,536,220,1343,0.7,1003,2.2
4606,7,2021-08-21,21439773999,91.18655746114226,Mauris Non Institute,"cable, satellite,...",a,6.10,511685,QLD,Female,0,52,1073,536,220,1343,0.7,1003,2.2
4606,7,2021-08-19,86662713230,38.8137172956379,Vestibulum Accums...,"watch, clock, and...",a,6.41,511685,QLD,Female,0,52,1073,536,220,1343,0.7,1003,2.2


In [64]:
# save dataset
full_dataset.write.mode('overwrite').parquet('../data/curated/full_dataset/')